In [42]:
import json
import numpy as np
import pandas as pd
import math
import ast

In [17]:
# with open('dataset/business_LV.json') as fd:
#     df = json.load(fd)
# print(len(df))
df = pd.read_csv('dataset/business_LV.csv')
print(len(df))

3923


In [ ]:
# ["business_id", "neighborhood", "stars", "review_count", 
#             "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups", 
#             "RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]

In [80]:
def variables(all_columns,mode):
    cols = all_columns
    cat_cols = ["neighborhood", "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]
    if mode == "cat":
        cols = ["neighborhood", "categories"]
    elif mode == "bool":
        cols = ["RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking"]
    elif mode == "cont":
        cols = ['stars', 'review_count']
    print(cols)
    return cols

In [53]:
# def format_long_data_cat(df,cols_to_feat,time):
    #First 6 hours
cols_to_feat = []
new_df = df.copy()
new_df = new_df.sort_values(['business_id'])
id_list = sorted(list(set(list(new_df['business_id']))))
cols = variables(new_df.columns,"cat")

for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        for col in cols:
            if (col == 'categories'):
                options = ast.literal_eval(row[col])
                for option in options:
                    if (option != "Restaurants" and option not in cols_to_feat):
                        cols_to_feat.append(option)
            else:
                suffix = row[col]
                if (type(suffix) is float and math.isnan(suffix)):
                    suffix = "missing"
                else:
                    suffix = row[col]
                if (col+'_'+suffix) not in cols_to_feat:
                    cols_to_feat.append(col+'_'+suffix)
# print(cols_to_feat)


#     ###CREATE DICTIONARY###
    
#     i = 0
#     for col in cols:
#         for j in range(i, i + 6):
#             if col == 'location':
#                 cols_to_feat[j] = col + location_suffix[j - i]
#             else:
#                 cols_to_feat[j] = col + suffix[j - i]
#         i += 6
    
#     for k,v in cols_to_feat.items():
#         if v == 'location_GarbageColumn':
#             del cols_to_feat[k]
#             break
#     ###CREATE DICTIONARY###
        
#     cat_arr = np.zeros((len(id_list),len(cols),6))
#     dic = {'LL':[1,0,0,0,0,0],'L':[0,1,0,0,0,0],'N':[0,0,1,0,0,0],'H':[0,0,0,1,0,0],'HH':[0,0,0,0,1,0],'A':[1,0,0,0,0,0]}
#     loc_dic = {'ED':[1,0,0,0,0,0],'ICU':[0,1,0,0,0,0],'floor':[0,0,1,0,0,0],'Procedure':[0,0,0,1,0,0]}
#     i = 0
#     for id in id_list:
#         temp = new_df3[new_df3['id']==id]
#         patient_feature_vector = np.zeros((len(cols),6))
#         j = 0
#         for c in cols:
#             column_vector = np.zeros(6)
#             for idx,row in temp.iterrows():
#                 if (c == 'location'):
#                     #Location is known
#                     if (not type(row[c]) is float):
#                         column_vector = np.logical_or(column_vector,loc_dic[row[c]]).astype(int)
#                     #Location is unknown
#                     else:
#                         column_vector[4] = 1
#                 #If continuous entry exists
#                 elif not math.isnan(row[c[:-5]]):
#                     #Normal level: N
#                     if (type(row[c]) is float):
#                         column_vector = np.logical_or(column_vector,dic['N']).astype(int)
#                     #Abnormal level: LL,L,H,HH
#                     else:
#                         column_vector = np.logical_or(column_vector,dic[row[c]]).astype(int)
#                 #If data is missing
#                 else:
#                     column_vector[5] =  1

#             if (sum(column_vector[:5]) and column_vector[5]):
#                 column_vector[5] = 0

#             patient_feature_vector[j] = column_vector.astype(int)
#             j += 1

#         cat_arr[i] = patient_feature_vector
#         i += 1
    
#     print('FINISHED CATEGORICAL VARIABLES!')
#     #Make it 2D
#     cat_arr = np.reshape(cat_arr,(cat_arr.shape[0],-1))
#     #Remove column of zeros caused by location
#     cat_arr = cat_arr[:,:-1]

#     return cat_arr,cols_to_feat

['neighborhood', 'categories']


In [69]:
cat_arr = np.zeros((len(id_list),len(cols_to_feat)))
print(cat_arr.shape)
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    feature_vector = np.zeros((len(cols_to_feat)))
    j = 0
    for col in cols:
        if (col == "categories"):
            options = ast.literal_eval(row[col])
            for option in options:
                if (option != "Restaurants"):
                    feature_vector[cols_to_feat.index(option)] = 1
        else:
            suffix = row[col]
            if (type(suffix) is float and math.isnan(suffix)):
                suffix = "missing"
            else:
                suffix = row[col]
            feature_vector[cols_to_feat.index(col+'_'+suffix)] = 1
    cat_arr[i] = feature_vector
    i+=1

(3923, 374)


In [75]:
cols = variables(new_df.columns,"bool")
for col in cols:
    cols_to_feat.append(col)
bool_arr = np.zeros((len(id_list), len(cols)))
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    feature_vector = np.zeros((len(cols)))
    j = 0
    for col in cols:
        if bool(row[col]):
            feature_vector[cols.index(col)] = 1
    bool_arr[i] = feature_vector
    i+=1
print(bool_arr[0])

['RestaurantsTakeOut', 'GoodForKids', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'DriveThru', 'WiFi', 'BusinessParking']
[ 1.  1.  1.  0.  0.  1.  1.]


In [81]:
cols= variables(new_df.columns, "cont")
li1 = []
for c in total_cols:
    for id in id_list:
        if not flag:
            li1.append(np.sum(new_df3[new_df3['id'] == id][c]))

['stars', 'review_count']
